In [1]:
lakehouseName = "RDS_IDM_test_LH_silver"

StatementMeta(, aa49e154-45d1-40f2-931a-4235807c533b, 3, Finished, Available)

In [13]:
# Helper functions

import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType, DoubleType, DecimalType, BinaryType
spark.conf.set('spark.sql.caseSensitive', True)

def remap(df, columnMap):
    dfRenamed = df.rename(columns=columnMap)
    dfRenamed = dfRenamed[columnMap.values()]
    df = spark.createDataFrame(dfRenamed) 
    return df

def renameDf(df, renameList):
    for col in renameList.keys():
        df = df.withColumnRenamed(col, renameList[col])
    return df

def retype(df, retypes):
    for col in retypes.keys():
        df = df.withColumn(col, df[col].cast(retypes[col]))
    return df

def save(df, tableName):
    print(f"Saving {tableName}")
    df  .write \
        .mode("overwrite") \
        .saveAsTable(F"{lakehouseName}.{tableName}")

def saveWithSchemaOverwrite(df, tableName):
    print(f"Saving {tableName}")
    df  .write \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(F"{lakehouseName}.{tableName}")

def saveWithSchemaMerge(df, tableName):
    print(f"Saving {tableName}")
    df  .write \
        .option("mergeSchema", "true") \
        .mode("overwrite") \
        .saveAsTable(F"{lakehouseName}.{tableName}")

StatementMeta(, aa49e154-45d1-40f2-931a-4235807c533b, 15, Finished, Available)

In [11]:
import pyspark.sql.functions as F

# Currency, Store, Location, Country and Order tables
dfCurrency = spark.read.format("csv").option("header", True).load("Files/IDM_Demo_Currency.csv")

dfLocation = spark.read.format("csv").option("header", True).load("Files/IDM_Demo_Location.csv")
colRetype={"LocationId":"int", "CountryId":"int"}
dfLocation = retype(dfLocation, colRetype);

dfCountry = spark.read.format("csv").option("header", True).load("Files/IDM_Demo_Country.csv")
colRetype={"CountryId":"int"}
dfCountry = retype(dfCountry, colRetype);

dfOrder = spark.read.format("csv").option("header", True).load("Files/IDM_Demo_Order.csv")
colRetype={"OrderId":"long", "OrderTotalAmount":"decimal", "OrderActualDeliveryTimestamp":"timestamp", "StoreId":"int", "OrderRecievedTimeStamp":"timestamp"}
dfOrder = retype(dfOrder, colRetype);

dfStore = spark.read.format("csv").option("header", True).load("Files/IDM_Demo_Store.csv")
colRetype={"StoreId":"int", "LocationId":"int", "StoreOpenDate":"date", "GrossFloorArea":"int", "MerchantId":"int"}
dfStore = retype(dfStore, colRetype);

# Persist normalized tables
saveWithSchemaMerge(dfCurrency,  "Currency")
saveWithSchemaMerge(dfLocation,  "Location")
saveWithSchemaMerge(dfCountry,  "Country")
saveWithSchemaMerge(dfOrder,  "Order")
saveWithSchemaMerge(dfStore,  "Store")

StatementMeta(, aa49e154-45d1-40f2-931a-4235807c533b, 13, Finished, Available)

Saving Currency
Saving Location
Saving Country
Saving Order
Saving Store


In [12]:
print(dfCurrency.count())
print(dfLocation.count())
print(dfCountry.count())
print(dfOrder.count())
print(dfStore.count())

StatementMeta(, aa49e154-45d1-40f2-931a-4235807c533b, 14, Finished, Available)

253
300
247
590
79
